# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Data Cleaning
from sklearn.impute import KNNImputer # to impute missing values using KNN imputation technique

# Dataset

In [3]:
file_path = 'ola_driver_scaler.csv'
_data = pd.read_csv(file_path)
_data.shape

(19104, 14)

- **MMMM-YY**: Reporting Date (Monthly)
- **Driver_ID**: Unique ID for drivers
- **Age**: Age of the driver
- **Gender**: Gender of the driver – **Male: 0**, **Female: 1**
- **City**: City code of the driver
- **Education_Level**: Education level – **0 = 10+**, **1 = 12+**, **2 = Graduate**
- **Income**: Monthly average income of the driver
- **Date Of Joining**: Joining date for the driver
- **LastWorkingDate**: Last date of working for the driver
- **Joining Designation**: Designation at the time of joining
- **Grade**: Grade of the driver at the time of reporting
- **Total Business Value**: Total business value acquired in a month (**negative = cancellations/refunds/EMI adjustments**)
- **Quarterly Rating**: Quarterly rating of the driver – **1 to 5 (higher is better)**


In [4]:
_data.head()

,Unnamed: 0,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1


- Unwanted column: Unnamed 0 ==> Dropping it.
- Monthly level data for each Driver_ID


Info from problem statement:-
- Target varable = LastWorkingDate: {
    null: not left,
    date: left
}

- Education: {
    10+: 0,
    12+: 1,
    graduate: 2
}

- Timeline: 2019-2020 (COVID)

In [5]:
# Dropping Unnamed: 0 column as it is not required for analysis
_data.drop('Unnamed: 0', axis=1, inplace=True)

# converting date columns to datetime format
_data['Dateofjoining'] = pd.to_datetime(_data['Dateofjoining'], format='%d/%m/%y')
_data['LastWorkingDate'] = pd.to_datetime(_data['LastWorkingDate'], format='%d/%m/%y')
_data['MMM-YY'] = pd.to_datetime(_data['MMM-YY'], format='%m/%d/%y')

In [6]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MMM-YY                19104 non-null  datetime64[ns]
 1   Driver_ID             19104 non-null  int64         
 2   Age                   19043 non-null  float64       
 3   Gender                19052 non-null  float64       
 4   City                  19104 non-null  object        
 5   Education_Level       19104 non-null  int64         
 6   Income                19104 non-null  int64         
 7   Dateofjoining         19104 non-null  datetime64[ns]
 8   LastWorkingDate       1616 non-null   datetime64[ns]
 9   Joining Designation   19104 non-null  int64         
 10  Grade                 19104 non-null  int64         
 11  Total Business Value  19104 non-null  int64         
 12  Quarterly Rating      19104 non-null  int64         
dtypes: datetime64[ns

In [7]:
_data.isna().sum()

MMM-YY                      0
Driver_ID                   0
Age                        61
Gender                     52
City                        0
Education_Level             0
Income                      0
Dateofjoining               0
LastWorkingDate         17488
Joining Designation         0
Grade                       0
Total Business Value        0
Quarterly Rating            0
dtype: int64

- Missing value Columns: Age, Gender
- LastWorkingDate is actually the Target Variable, hence not the Missing value column

**Handling LastWorkingDate Column**

In [8]:
# latest_working_date = _data['LastWorkingDate'].max()
# latest_working_date

In [9]:
# # adding one day to latest_working_date to get the next date after last_date
# latest_working_date = latest_working_date + pd.Timedelta(days=1)
# latest_working_date

In [10]:
# _data['LastWorkingDate'] = _data['LastWorkingDate'].fillna(latest_working_date)
# _data.head()

# EDA

## kNN Imputation on Missing values (Age, Gender)

In [11]:
_data_nums = _data.select_dtypes(include=np.number)
_data_nums.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,28.0,0.0,2,57387,1,1,2381060,2
1,1,28.0,0.0,2,57387,1,1,-665480,2
2,1,28.0,0.0,2,57387,1,1,0,2
3,2,31.0,0.0,2,67016,2,2,0,1
4,2,31.0,0.0,2,67016,2,2,0,1


In [12]:
# dropping DriverID column (not required for kNN imputation)
_data_nums.drop('Driver_ID', axis=1, inplace=True)

columns = _data_nums.columns

In [13]:
knn_imputer = KNNImputer(n_neighbors=5) # imputer object
_data_nums_imputed = knn_imputer.fit_transform(_data_nums) # imputed data in numpy array format
_data_nums_imputed

array([[ 2.80000e+01,  0.00000e+00,  2.00000e+00, ...,  1.00000e+00,
         2.38106e+06,  2.00000e+00],
       [ 2.80000e+01,  0.00000e+00,  2.00000e+00, ...,  1.00000e+00,
        -6.65480e+05,  2.00000e+00],
       [ 2.80000e+01,  0.00000e+00,  2.00000e+00, ...,  1.00000e+00,
         0.00000e+00,  2.00000e+00],
       ...,
       [ 3.00000e+01,  0.00000e+00,  2.00000e+00, ...,  2.00000e+00,
         0.00000e+00,  2.00000e+00],
       [ 3.00000e+01,  0.00000e+00,  2.00000e+00, ...,  2.00000e+00,
         2.00420e+05,  2.00000e+00],
       [ 3.00000e+01,  0.00000e+00,  2.00000e+00, ...,  2.00000e+00,
         4.11480e+05,  2.00000e+00]], shape=(19104, 8))

In [14]:
_data_nums_imputed = pd.DataFrame(_data_nums_imputed, columns=columns) # converting imputed data to dataframe format
_data_nums_imputed.head()

,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,28.0,0.0,2.0,57387.0,1.0,1.0,2381060.0,2.0
1,28.0,0.0,2.0,57387.0,1.0,1.0,-665480.0,2.0
2,28.0,0.0,2.0,57387.0,1.0,1.0,0.0,2.0
3,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,1.0
4,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,1.0


**get remaining columns back**

In [15]:
remaining_columns = list(set(_data.columns) - set(_data_nums.columns))
remaining_columns

['LastWorkingDate', 'MMM-YY', 'Driver_ID', 'Dateofjoining', 'City']

In [16]:
# concat

data = pd.concat([_data_nums_imputed, _data[remaining_columns]], axis=1)
data.head()

,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating,LastWorkingDate,MMM-YY,Driver_ID,Dateofjoining,City
0,28.0,0.0,2.0,57387.0,1.0,1.0,2381060.0,2.0,NaT,2019-01-01,1,2018-12-24,C23
1,28.0,0.0,2.0,57387.0,1.0,1.0,-665480.0,2.0,NaT,2019-02-01,1,2018-12-24,C23
2,28.0,0.0,2.0,57387.0,1.0,1.0,0.0,2.0,2019-11-03,2019-03-01,1,2018-12-24,C23
3,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,1.0,NaT,2020-11-01,2,2020-06-11,C7
4,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,1.0,NaT,2020-12-01,2,2020-06-11,C7


In [17]:
data.columns

Index(['Age', 'Gender', 'Education_Level', 'Income', 'Joining Designation',
       'Grade', 'Total Business Value', 'Quarterly Rating', 'LastWorkingDate',
       'MMM-YY', 'Driver_ID', 'Dateofjoining', 'City'],
      dtype='object')

Sanity check to **verify correct concatenation**

In [18]:
_data['Driver_ID'].unique()

array([   1,    2,    4, ..., 2786, 2787, 2788], shape=(2381,))

In [19]:
id = 2786

_data.loc[_data['Driver_ID'] == id]

,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
19082,2019-01-01,2786,44.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,221080,2
19083,2019-02-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,485270,2
19084,2019-03-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,970380,2
19085,2019-04-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,432240,2
19086,2019-05-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,387660,2
19087,2019-06-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,0,2
19088,2019-07-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,318460,1
19089,2019-08-01,2786,45.0,0.0,C19,0,35370,2018-07-31,NaT,2,2,0,1
19090,2019-09-01,2786,45.0,0.0,C19,0,35370,2018-07-31,2019-09-22,2,2,0,1


In [20]:
data.loc[data['Driver_ID'] == id]

,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating,LastWorkingDate,MMM-YY,Driver_ID,Dateofjoining,City
19082,44.0,0.0,0.0,35370.0,2.0,2.0,221080.0,2.0,NaT,2019-01-01,2786,2018-07-31,C19
19083,45.0,0.0,0.0,35370.0,2.0,2.0,485270.0,2.0,NaT,2019-02-01,2786,2018-07-31,C19
19084,45.0,0.0,0.0,35370.0,2.0,2.0,970380.0,2.0,NaT,2019-03-01,2786,2018-07-31,C19
19085,45.0,0.0,0.0,35370.0,2.0,2.0,432240.0,2.0,NaT,2019-04-01,2786,2018-07-31,C19
19086,45.0,0.0,0.0,35370.0,2.0,2.0,387660.0,2.0,NaT,2019-05-01,2786,2018-07-31,C19
19087,45.0,0.0,0.0,35370.0,2.0,2.0,0.0,2.0,NaT,2019-06-01,2786,2018-07-31,C19
19088,45.0,0.0,0.0,35370.0,2.0,2.0,318460.0,1.0,NaT,2019-07-01,2786,2018-07-31,C19
19089,45.0,0.0,0.0,35370.0,2.0,2.0,0.0,1.0,NaT,2019-08-01,2786,2018-07-31,C19
19090,45.0,0.0,0.0,35370.0,2.0,2.0,0.0,1.0,2019-09-22,2019-09-01,2786,2018-07-31,C19


Same datapoints being fetched for id=2876, (before and after, same indices)
Hence Correct concat

## Aggregation

In [21]:
data.sort_values(by=['Driver_ID', 'MMM-YY'], inplace=True)
data.head(10)

,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating,LastWorkingDate,MMM-YY,Driver_ID,Dateofjoining,City
0,28.0,0.0,2.0,57387.0,1.0,1.0,2381060.0,2.0,NaT,2019-01-01,1,2018-12-24,C23
1,28.0,0.0,2.0,57387.0,1.0,1.0,-665480.0,2.0,NaT,2019-02-01,1,2018-12-24,C23
2,28.0,0.0,2.0,57387.0,1.0,1.0,0.0,2.0,2019-11-03,2019-03-01,1,2018-12-24,C23
3,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,1.0,NaT,2020-11-01,2,2020-06-11,C7
4,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,1.0,NaT,2020-12-01,2,2020-06-11,C7
5,43.0,0.0,2.0,65603.0,2.0,2.0,0.0,1.0,NaT,2019-12-01,4,2019-07-12,C13
6,43.0,0.0,2.0,65603.0,2.0,2.0,0.0,1.0,NaT,2020-01-01,4,2019-07-12,C13
7,43.0,0.0,2.0,65603.0,2.0,2.0,0.0,1.0,NaT,2020-02-01,4,2019-07-12,C13
8,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,1.0,NaT,2020-03-01,4,2019-07-12,C13
9,43.0,0.0,2.0,65603.0,2.0,2.0,0.0,1.0,2020-04-27,2020-04-01,4,2019-07-12,C13


In [22]:
agg_dict = {
    'Age': 'last',                           # or max
    'Gender': 'first',                       # not changing
    'Education_Level': 'last',               # or max
    'Income': 'last',                        # latest income is more relevant
    'Joining Designation': 'first',          # not changing
    'Grade': 'last',                         # latest grade is more relevant
    'Total Business Value': 'sum',           # sum of business value across months is more relevant
    'LastWorkingDate': 'max',                # latest date is more relevant
    'Quarterly Rating': 'last',              # latest rating is more relevant
    'Dateofjoining': 'min',                  # not changing
    'City': 'last',                          # latest city is more relevant
    'MMM-YY': 'last'                         # latest month is more relevant

}

data_agg = data.groupby('Driver_ID').agg(agg_dict).reset_index()
data_agg.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,LastWorkingDate,Quarterly Rating,Dateofjoining,City,MMM-YY
0,1,28.0,0.0,2.0,57387.0,1.0,1.0,1715580.0,2019-11-03,2.0,2018-12-24,C23,2019-03-01
1,2,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,NaT,1.0,2020-06-11,C7,2020-12-01
2,4,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,2020-04-27,1.0,2019-07-12,C13,2020-04-01
3,5,29.0,0.0,0.0,46368.0,1.0,1.0,120360.0,2019-07-03,1.0,2019-09-01,C9,2019-03-01
4,6,31.0,1.0,1.0,78728.0,3.0,3.0,1265000.0,NaT,2.0,2020-07-31,C11,2020-12-01


### Feature Engineering

#### 1. Creating a column which tells if the quarterly rating has increased for that employee for those whose quarterly rating has increased we assign the value 1

In [23]:
data.groupby('Driver_ID')['Quarterly Rating'].agg(['first', 'last']).assign(**{'Quarterly Rating Increased': lambda row: row['last'] > row['first']}).reset_index()[['Driver_ID', 'Quarterly Rating Increased']]

,Driver_ID,Quarterly Rating Increased
0,1,False
1,2,False
2,4,False
3,5,False
4,6,True
...,...,...
2376,2784,True
2377,2785,False
2378,2786,False
2379,2787,False


In [24]:
test = data.groupby('Driver_ID')['Quarterly Rating'].agg(['first', 'last'])
test = test.assign(**{'Quarterly Rating Increased': lambda row: row['last'] > row['first']})
test = test.reset_index()[['Driver_ID', 'Quarterly Rating Increased']]
test['Quarterly Rating Increased'] = test['Quarterly Rating Increased'].astype(int)
test

,Driver_ID,Quarterly Rating Increased
0,1,0
1,2,0
2,4,0
3,5,0
4,6,1
...,...,...
2376,2784,1
2377,2785,0
2378,2786,0
2379,2787,0


In [25]:
data_agg = data_agg.merge(test, on='Driver_ID', how='left')
data_agg.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,LastWorkingDate,Quarterly Rating,Dateofjoining,City,MMM-YY,Quarterly Rating Increased
0,1,28.0,0.0,2.0,57387.0,1.0,1.0,1715580.0,2019-11-03,2.0,2018-12-24,C23,2019-03-01,0
1,2,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,NaT,1.0,2020-06-11,C7,2020-12-01,0
2,4,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,2020-04-27,1.0,2019-07-12,C13,2020-04-01,0
3,5,29.0,0.0,0.0,46368.0,1.0,1.0,120360.0,2019-07-03,1.0,2019-09-01,C9,2019-03-01,0
4,6,31.0,1.0,1.0,78728.0,3.0,3.0,1265000.0,NaT,2.0,2020-07-31,C11,2020-12-01,1


#### 2. Creating a column which tells whether the monthly income has increased for that driver - for those whose monthly income has increased we assign the value 1

In [26]:
test = data.groupby('Driver_ID')['Income'].agg(['first', 'last'])
test = test.assign(**{'Income Increased': lambda row: row['last'] > row['first']})
test = test.reset_index()[['Driver_ID', 'Income Increased']]
test['Income Increased'] = test['Income Increased'].astype(int)
test

,Driver_ID,Income Increased
0,1,0
1,2,0
2,4,0
3,5,0
4,6,0
...,...,...
2376,2784,0
2377,2785,0
2378,2786,0
2379,2787,0


In [27]:
data_agg = data_agg.merge(test, on='Driver_ID', how='left')
data_agg.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,LastWorkingDate,Quarterly Rating,Dateofjoining,City,MMM-YY,Quarterly Rating Increased,Income Increased
0,1,28.0,0.0,2.0,57387.0,1.0,1.0,1715580.0,2019-11-03,2.0,2018-12-24,C23,2019-03-01,0,0
1,2,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,NaT,1.0,2020-06-11,C7,2020-12-01,0,0
2,4,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,2020-04-27,1.0,2019-07-12,C13,2020-04-01,0,0
3,5,29.0,0.0,0.0,46368.0,1.0,1.0,120360.0,2019-07-03,1.0,2019-09-01,C9,2019-03-01,0,0
4,6,31.0,1.0,1.0,78728.0,3.0,3.0,1265000.0,NaT,2.0,2020-07-31,C11,2020-12-01,1,0


In [28]:
data_agg['Income Increased'].value_counts()

Income Increased
0    2338
1      43
Name: count, dtype: int64

#### 3. Target variable creation: Creating a column called Churn which tells whether the driver has left the company- driver whose last working day is present will have the value 1

In [31]:
data_agg.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,LastWorkingDate,Quarterly Rating,Dateofjoining,City,MMM-YY,Quarterly Rating Increased,Income Increased
0,1,28.0,0.0,2.0,57387.0,1.0,1.0,1715580.0,2019-11-03,2.0,2018-12-24,C23,2019-03-01,0,0
1,2,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,NaT,1.0,2020-06-11,C7,2020-12-01,0,0
2,4,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,2020-04-27,1.0,2019-07-12,C13,2020-04-01,0,0
3,5,29.0,0.0,0.0,46368.0,1.0,1.0,120360.0,2019-07-03,1.0,2019-09-01,C9,2019-03-01,0,0
4,6,31.0,1.0,1.0,78728.0,3.0,3.0,1265000.0,NaT,2.0,2020-07-31,C11,2020-12-01,1,0


**first creating a helper column LastWorkingDate2**

- if LastWorkingDate is None then LastWorkingDate2 = latest_data_date
- else LastWorkingDate2 = LastWorkingDate

In [32]:
latest_data_date = max(data_agg['LastWorkingDate'].max(), data_agg['Dateofjoining'].max(), data_agg['MMM-YY'].max())
latest_data_date = latest_data_date + pd.Timedelta(days=1)
latest_data_date

Timestamp('2020-12-29 00:00:00')

In [33]:
data_agg['LastWorkingDate2'] = data_agg['LastWorkingDate'].fillna(latest_data_date)
data_agg.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,LastWorkingDate,Quarterly Rating,Dateofjoining,City,MMM-YY,Quarterly Rating Increased,Income Increased,LastWorkingDate2
0,1,28.0,0.0,2.0,57387.0,1.0,1.0,1715580.0,2019-11-03,2.0,2018-12-24,C23,2019-03-01,0,0,2019-11-03
1,2,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,NaT,1.0,2020-06-11,C7,2020-12-01,0,0,2020-12-29
2,4,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,2020-04-27,1.0,2019-07-12,C13,2020-04-01,0,0,2020-04-27
3,5,29.0,0.0,0.0,46368.0,1.0,1.0,120360.0,2019-07-03,1.0,2019-09-01,C9,2019-03-01,0,0,2019-07-03
4,6,31.0,1.0,1.0,78728.0,3.0,3.0,1265000.0,NaT,2.0,2020-07-31,C11,2020-12-01,1,0,2020-12-29


Now **Churn** Column

In [34]:
test = latest_data_date - data_agg['LastWorkingDate2']
test = test.dt.days
test

0       422
1         0
2       246
3       545
4         0
       ... 
2376      0
2377     62
2378    464
2379    558
2380      0
Name: LastWorkingDate2, Length: 2381, dtype: int64

In [35]:
data_agg['Churn'] = np.int64(test > 0)
data_agg.head()

,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,LastWorkingDate,Quarterly Rating,Dateofjoining,City,MMM-YY,Quarterly Rating Increased,Income Increased,LastWorkingDate2,Churn
0,1,28.0,0.0,2.0,57387.0,1.0,1.0,1715580.0,2019-11-03,2.0,2018-12-24,C23,2019-03-01,0,0,2019-11-03,1
1,2,31.0,0.0,2.0,67016.0,2.0,2.0,0.0,NaT,1.0,2020-06-11,C7,2020-12-01,0,0,2020-12-29,0
2,4,43.0,0.0,2.0,65603.0,2.0,2.0,350000.0,2020-04-27,1.0,2019-07-12,C13,2020-04-01,0,0,2020-04-27,1
3,5,29.0,0.0,0.0,46368.0,1.0,1.0,120360.0,2019-07-03,1.0,2019-09-01,C9,2019-03-01,0,0,2019-07-03,1
4,6,31.0,1.0,1.0,78728.0,3.0,3.0,1265000.0,NaT,2.0,2020-07-31,C11,2020-12-01,1,0,2020-12-29,0


In [36]:
data_agg['Churn'].value_counts()

Churn
1    1616
0     765
Name: count, dtype: int64

In [37]:
data_agg.to_csv('ola_driver_preprocess_agg.csv', index=False)